# 8. Switching to PyTorch
> Comparing the performance of PyTorch and NumPy based Pagels lambda

In [8]:
# All the imports
%load_ext autoreload
%autoreload 2

import ete3
import numpy as np
import pandas as pd
import torch
from matplotlib import pyplot as plt

# Stuff for timing
import time
from tqdm import tqdm

# My stuff
from src.ihmp import get_diffs
from src.phylogenetic_signal_torch import PagelsLambda as PagelsLambdaTorch
from src.phylogenetic_signal import PagelsLambda as PagelsLambdaNumpy

treepath = "greengenes/data/gg_13_5_otus_99_annotated.tree"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Get data and prune tree appropriately

ibd = get_diffs("ibd")

tree = ete3.Tree(treepath, format=1, quoted_node_names=True)
leaves = ibd.columns
tree.prune(leaves)

ibd.head()

1000269    1008348  1009894  1012376   1017181  \
site  patient visit                                                     
feces 3002    6     -10.298141 -10.991272      0.0      0.0 -11.68441   
              8       0.000000   0.000000      0.0      0.0   0.00000   
              9       0.000000   0.000000      0.0      0.0   0.00000   
      3003    5       0.000000   0.000000      0.0      0.0   0.00000   
              6       0.000000   0.000000      0.0      0.0   0.00000   

                      1017413  1019823    1019878  102222  1023075  ...  \
site  patient visit                                                 ...   
feces 3002    6      11.70451      0.0   0.307779     0.0      0.0  ...   
              8     -11.70451      0.0 -11.704510     0.0      0.0  ...   
              9      10.83337      0.0   0.000000     0.0      0.0  ...   
      3003    5       0.00000      0.0   0.000000     0.0      0.0  ...   
              6       0.00000      0.0   0.000000     0.0      0.0  ...   

                     964363     968675     968954     971907     975306  \
site  patient visit                                                       
feces 3002    6         0.0  -0.028690  -2.331253 -12.244023  -1.152620   
              8         0.0 -13.313941 -10.605914   0.000000 -14.557134   
              9         0.0  12.219649  11.526507  10.833370  14.704551   
      3003    5         0.0   0.000000   0.000000   0.000000   0.000000   
              6         0.0  11.768101   0.000000   0.000000   0.000000   

                        976470  979707     988375     988932     999046  
site  patient visit                                                      
feces 3002    6     -11.396731     0.0 -14.059309   1.224061 -10.298141  
              8       0.000000     0.0   0.000000 -12.215332   0.000000  
              9       0.000000     0.0   0.000000  11.526507   0.000000  
      3003    5       0.000000     0.0   0.000000   0.000000   0.000000  
              6       0.000000     0.0  15.026190  11.768101   0.000000  

[5 rows x 1370 columns]

In [33]:
# Time initialization - this is probably not the slowest part of the code
# At least you only have to do it once

t1 = time.time()
pl_torch = PagelsLambdaTorch(tree)
t2 = time.time()
print(f"Time to initialize PagelsLambdaTorch: {t2-t1:.2f} seconds")

# pl_numpy = PagelsLambdaNumpy(tree)
# t3 = time.time()
# print(f"Time to initialize PagelsLambdaNumpy: {t3-t2:.2f} seconds")

Time to initialize PagelsLambdaTorch: 70.47 seconds


In [34]:
# Free preprocessing: turn df into Tensor

ibd_torch = torch.tensor(ibd.values, dtype=torch.float32)

t1 = time.time()
torch_lams = []
for i in tqdm(range(10)):
    pl_torch.fit(ibd_torch[i])
    torch_lams.append(pl_torch.lam)
t2 = time.time()
print(f"Time to fit PagelsLambdaTorch: {t2-t1:.2f} seconds")

# np_lams = []
# for idx in tqdm(range(10)):
#     pl_numpy.fit(ibd.iloc[idx].values)
#     np_lams.append(pl_numpy.lam)
# t3 = time.time()
# print(f"Time to fit PagelsLambdaNumpy: {t3-t2:.2f} seconds")

# # Compare results
# diffs = np.array(torch_lams) - np.array(np_lams)
# print(f"Total difference: {np.sum(diffs):.2f}")
print(np.array(torch_lams))
# print(np.array(np_lams))

 10%|█         | 1/10 [00:25<03:51, 25.73s/it]

-0.03923378139734268 tensor([[4240.7456]], grad_fn=<NegBackward0>)
0.15255992114543915 tensor([[4230.9814]], grad_fn=<NegBackward0>)
0.15255992114543915 tensor([[4227.0430]], grad_fn=<NegBackward0>)


 20%|██        | 2/10 [01:15<05:00, 37.60s/it]


Intel MKL ERROR: Parameter 4 was incorrect on entry to SLASCL.

Intel MKL ERROR: Parameter 4 was incorrect on entry to SLASCL.


_LinAlgError: linalg.svd: The algorithm failed to converge because the input matrix is ill-conditioned or has too many repeated singular values (error code: 20).